In [1]:
######################################################
### Attaching and downloading necessary packagaes ####
######################################################
# if (!requireNamespace("BiocManager", quietly = FALSE))
#     install.packages("BiocManager")
# BiocManager::version()
# library("devtools")
# devtools::install_github("broadinstitute/infercnv", force=TRUE)
# detach("package:infercnv", unload=T)  ### package infercnv_1.15.3  seems to work fine
library(infercnv)
#### loading packages and others 
library("Seurat")
library("tidyverse")
#remotes::install_github("mojaveazure/seurat-disk")
library("SeuratDisk")
#install.packages("ggplot2")
library(ggplot2)
#install.packages("remotes")
#remotes::install_github("chris-mcginnis-ucsf/DoubletFinder", force=TRUE)
library(DoubletFinder)
library(dplyr)
library(tidyverse)

Attaching SeuratObject

-- Attaching core tidyverse packages ------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse 2.0.0 --
v dplyr     1.1.1     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.2     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.1     
-- Conflicts ------------------------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [2]:
####################################################################
#### Working with saved object to create InferCNV plots and so #####
####################################################################

#### trying working with seurat object ---
## trying InferCNV from the saved .rds file
SCAF2221 <- readRDS("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/SCAF2221_cell_cycle_regress_difference_out.rds")
load("/data/kumarr9/scRNA/PBMC_seu.RData")
## step 16. mergind the two seurat object 
# link - https://satijalab.org/seurat/articles/merge_vignette.html
# We are merging based on normalized one and hence added the parameters of merge.data=TRUE
SCAF2221_merged <- merge(SCAF2221, y = PBMC_seu, add.cell.ids = c("SCLC", "PBMC"), project = "SCLC",
     merge.data = TRUE)
# to see the content of merged object
# SCAF3163_merged@assays$RNA@counts
# dim(SCAF3163_merged)
# dim(SCAF3163)
# dim(PBMC_seu)

## step 17. Getting count matrix from merged dataframe
#str(SCAF3163_merged)
expr_matrix <- GetAssayData(object = SCAF2221_merged, slot = "counts", assay = "RNA")
# converting the expr_matrix which is dgCmatrix to dataframe 
expr_df <- as.data.frame(expr_matrix)
#expr_df[1:20, 1:20]
write.table(expr_df, file = "/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/SCAF_matrix.csv", sep = "\t", row.names = TRUE, col.names = TRUE)
### creating gene order matrix for infercnv 
gene_names <- rownames(SCAF2221_merged)
### converted it to dataframe 
gene_names <- data.frame(gene_names)
## adding chromosme number, start and end to it
## read a gtf file ----
gtf_file <- read.table("/data/SCLCgenomics/rajesh/gene_version_27_cp.gtf")
matching_rows <- gtf_file[gtf_file$V5 %in% gene_names$gene_names, ]
### correcting this dataframe as given in inferCNV package
gene_df <- matching_rows %>% 
  select(V5, everything())
gene_df <- gene_df[, -5]
gene_df = gene_df[!duplicated(gene_df$V5),]
# names(gene_df)[2] <- "chr"
# names(gene_df)[3] <- "start"
# names(gene_df)[4] <- "stop"

###
write.table(gene_df, file = "/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/gene_order.txt", sep = "\t", row.names = FALSE, col.names = FALSE)
## getting the cluster info from the seurat object/merged object
clust_info <- SCAF2221_merged@meta.data[, "seurat_clusters", drop=F]
## making dataset what looks like or needed by the inferCNV
clust_info$cellID <- rownames(clust_info)
clust_info$clstr <- paste0('cluster.', clust_info$seurat_clusters)
clust_info_df <- clust_info[, !names(clust_info) %in% "seurat_clusters"]
write.table(clust_info_df, file = "/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/annotation.txt", sep = "\t", row.names = FALSE, col.names = FALSE)


Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 2.4 GiB"


In [4]:
# creating infercnv object
## Modification you need to do before creating object --
#1. Open the annotation.txt file and replace the PBMC cluster with PBMC only 
SCAF2221_infercnv_obj = CreateInfercnvObject(raw_counts_matrix="/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/SCAF_matrix.csv",
                                    annotations_file="/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/annotation.txt",
                                    gene_order_file="/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/gene_order.txt",
                                    ref_group_names=c("PBMC"))

INFO [2023-05-18 09:18:40] Parsing matrix: /data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/SCAF_matrix.csv
INFO [2023-05-18 09:19:44] Parsing gene order file: /data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/gene_order.txt
INFO [2023-05-18 09:19:44] Parsing cell annotations file: /data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/annotation.txt
INFO [2023-05-18 09:19:44] ::order_reduce:Start.
INFO [2023-05-18 09:19:45] .order_reduce(): expr and order match.
INFO [2023-05-18 09:19:47] ::process_data:order_reduce:Reduction from positional data, new dimensions (r,c) = 59890,5455 Total=18095240 Min=0 Max=3089.
INFO [2023-05-18 09:19:48] num genes removed taking into account provided gene ordering list: 6262 = 10.4558356987811% removed.
INFO [2023-05-18 09:19:48] -filtering out cells < 100 or > Inf, removing 0 % of cells
WARN [2023-05-18 09:19:50] Please use "options(scipen = 100)" before running infercnv if you

In [5]:
# perform infercnv operations to reveal cnv signal
infercnv_obj4 = infercnv::run(SCAF2221_infercnv_obj,
                             cutoff=0.1,  # use 1 for smart-seq, 0.1 for 10x-genomics
                             out_dir="/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/infercnv_results",  # dir is auto-created for storing outputs
                             cluster_by_groups=T,   # cluster
                             denoise=T,
                             HMM=T
                             )

INFO [2023-05-18 09:31:33] ::process_data:Start
INFO [2023-05-18 09:31:33] Checking for saved results.
INFO [2023-05-18 09:31:33] 

	STEP 1: incoming data

INFO [2023-05-18 09:32:06] 

	STEP 02: Removing lowly expressed genes

INFO [2023-05-18 09:32:06] ::above_min_mean_expr_cutoff:Start
INFO [2023-05-18 09:32:07] Removing 49887 genes from matrix as below mean expr threshold: 0.1
INFO [2023-05-18 09:32:08] validating infercnv_obj
INFO [2023-05-18 09:32:08] There are 3741 genes and 5455 cells remaining in the expr matrix.
INFO [2023-05-18 09:32:09] no genes removed due to min cells/gene filter
INFO [2023-05-18 09:32:16] 

	STEP 03: normalization by sequencing depth

INFO [2023-05-18 09:32:16] normalizing counts matrix by depth
INFO [2023-05-18 09:32:16] Computed total sum normalization factor as median libsize: 2567.000000
INFO [2023-05-18 09:32:16] Adding h-spike
INFO [2023-05-18 09:32:16] -hspike modeling of PBMC
INFO [2023-05-18 09:33:24] validating infercnv_obj
INFO [2023-05-18 09:3

Centering and scaling data matrix

PC_ 1 
Positive:  DNM2, ILF3, ILF3-AS1, TMEM205, SLC44A2, CDKN2D, PRKCSH, CDC37, ICAM3, ELOF1 
	   MRPL4, ACP5, DNMT1, EIF3G, C19orf66, RPL10P16, PIN1, UBL5, MYO1F, WDR83OS 
	   RAB8A, FAM32A, AP1M1, DHPS, TPM4, HNRNPM, TRIR, KLF2, RASAL3, SMIM7 
Negative:  IFI6, THEMIS2, FGR, RPA2, WASF2, SYTL1, DNAJC8, NUDC, ATPIF1, ARID1A 
	   SNHG3, RPS6KA1, SNHG12, HMGN2, TAF12, AL513365.1, YTHDF2, CD52, EPB41, SH3BGRL3 
	   ZNF593, SRSF4, STMN1, LAPTM5, LDLRAP1, PUM1, TMEM50A, SNRNP40, RSRP1, ZCCHC17 
PC_ 2 
Positive:  PF4, CXCL8, PPBP, DCK, CXCL3, GRSF1, CXCL2, JCHAIN, YTHDC1, AREG 
	   CENPC, G3BP2, IGFBP7, USO1, POLR2B, NAAA, REST, SDAD1, HOPX, SCARB2 
	   SRP72, CCNI, CNOT6L, NMU, MRPL1, TMEM165, TM9SF3, LINC01088, PDLIM1, FIP1L1 
Negative:  USF2, LSR, TMEM147, FXYD5, RBM42, GRAMD1A, COX6B1, ZNF302, U2AF1L4, PSENEN 
	   UBA2, GPI, HCST, TYROBP, POLR2I, LSM14A, TBCB, PEPD, CEBPG, PDCD5 
	   CAPNS1, URI1, POP4, SPINT2, UQCRFS1, RPSAP58, YIF1B, ZNF101, NDUFA13,

INFO [2023-05-18 09:37:44] define_signif_tumor_subclusters(), tumor: cluster.1


Centering and scaling data matrix

PC_ 1 
Positive:  MRPL34, RAB8A, DDA1, TPM4, BABAM1, FAM32A, AP1M1, RASAL3, BST2, USE1 
	   KLF2, SMIM7, BRD4, MYO9B, PGLS, NDUFB7, TECR, RPL18A, DNAJB1, CCDC124 
	   PKN1, DDX39A, IFI30, ADGRE5, ASF1B, IL27RA, JUND, C19orf53, LSM4, IER2 
Negative:  CCNL1, RSRC1, MLF1, MFSD1, TIPARP, SMC4, SSR3, KPNA4, GMPS, DHX36 
	   NMD3, RAP2B, RPL23AP42, MBNL1, PDCD10, SIAH2, SELENOT, AC074033.1, EIF2A, PFN2 
	   SERP1, RNF13, COMMD2, SEC62, TM4SF1, PHC3, HLTF, GYG1, CLDN11, PLSCR1 
PC_ 2 
Positive:  PRKRA, NFE2L2, UBE2E3, HNRNPA3, ITGA4, AC016739.1, ZC3H15, ATP5G3, SLC40A1, WIPF1 
	   CIR1, ORMDL1, OLA1, TLK1, AC007969.1, GORASP2, METTL5, DYNC1I2, HAT1, SP3 
	   ACADM, SSB, ZRANB2, RABGGTB, GLS, PPIG, USP33, SRSF11, AC107075.1, FUBP1 
Negative:  PPIA, TMED4, AC004453.1, H2AFV, POLD2, MYO1G, DBNL, MRPS24, SNHG15, BLVRA 
	   CCM2, COA1, STK17A, UPP1, MRPL32, IKZF1, SEC61G, PSMA2, VOPP1, MPLKIP 
	   NIPSNAP2, CCT6A, SUMF2, CHCHD2, RALA, GUSB, RPL35P5, TRGC2, TMEM24

INFO [2023-05-18 09:37:44] define_signif_tumor_subclusters(), tumor: cluster.10


...

Centering and scaling data matrix

PC_ 1 
Positive:  AATF, GGNBP2, SYNRG, ZNHIT3, MLLT6, CCL4, CISD3, CCL3, RPL19, CCL14 
	   CWC25, PSMB3, RPL23, LASP1, SLFN5, CCL5, TAF15, AP2B1, CDK12, MIEN1 
	   IKZF3, PSMD3, CASC3, TOP2A, IGFBP4, CCR7, SMARCE1, KRT10, KRT15, EIF1 
Negative:  RAB25, LMNA, PMF1, CCT3, NAXE, CRABP2, ISG20L2, MRPL24, HDGF, PRCC 
	   MNDA, PYHIN1, IFI16, C20orf24, NORAD, SAMHD1, SCAND1, RPN2, PHF20, ROMO1 
	   RBM39, CPNE1, BLCAP, ERGIC3, EIF6, MMP24-AS1, SNHG17, GSS, DYNLRB1, AHCY 
PC_ 2 
Positive:  CTSW, FIBP, CFL1, CCDC85B, DRAP1, RNASEH2C, SART1, BANF1, RELA, SF3B2 
	   SIPA1, RAB1B, YIF1A, MALAT1, BRMS1, NEAT1, MRPL11, DPF2, CTSF, RBM14 
	   CAPN1, RBM4, FAU, KDM2A, VPS51, GRK2, ARL2, POLD4, EHD1, PPP1CA 
Negative:  AC074033.1, PDCD10, RPL23AP42, SEC62, PHC3, NMD3, CLDN11, KPNA4, RPL22L1, TNFSF10 
	   AC069218.1, SMC4, TBL1XR1, MFSD1, ZNF639, ACTL6A, MLF1, MRPL47, RSRC1, NDUFB5 
	   CCNL1, TTC14, TIPARP, FXR1, SSR3, DNAJC19, GMPS, KLHL24, DHX36, PARL 
PC_ 3 
Posit

INFO [2023-05-18 09:38:00] -mirroring for hspike
INFO [2023-05-18 09:38:00] define_signif_tumor_subclusters(p_val=0.1
INFO [2023-05-18 09:38:00] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_PBMC
INFO [2023-05-18 09:38:00] cut tree into: 1 groups
INFO [2023-05-18 09:38:00] -processing spike_tumor_cell_PBMC,spike_tumor_cell_PBMC_s1
INFO [2023-05-18 09:38:00] define_signif_tumor_subclusters(), tumor: simnorm_cell_PBMC
INFO [2023-05-18 09:38:00] cut tree into: 1 groups
INFO [2023-05-18 09:38:00] -processing simnorm_cell_PBMC,simnorm_cell_PBMC_s1
INFO [2023-05-18 09:38:20] ::plot_cnv:Start
INFO [2023-05-18 09:38:20] ::plot_cnv:Current data dimensions (r,c)=3741,5455 Total=20487540.1331447 Min=0.621501379640084 Max=2.64071586976706.
INFO [2023-05-18 09:38:21] ::plot_cnv:Depending on the size of the matrix this may take a moment.
INFO [2023-05-18 09:38:22] plot_cnv(): auto thresholding at: (0.782715 , 1.225163)
INFO [2023-05-18 09:38:31] plot_cnv_observation:Start
INFO [2023-05-

INFO [2023-05-18 09:40:32] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-05-18 09:40:32] Quantiles of plotted data range: 0.782714933582637,0.94900337688813,0.998365237918168,1.05552958468344,1.22516319868467
INFO [2023-05-18 09:40:37] plot_cnv_references:Start
INFO [2023-05-18 09:40:37] Reference data size: Cells= 3372 Genes= 3741
INFO [2023-05-18 09:44:17] plot_cnv_references:Number reference groups= 1
INFO [2023-05-18 09:44:17] plot_cnv_references:Plotting heatmap.
INFO [2023-05-18 09:44:20] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-05-18 09:44:20] Quantiles of plotted data range: 0.782714933582637,0.975163427706493,0.99836518039734,1.02336730126517,1.22516319868467
INFO [2023-05-18 09:44:24] 

	STEP 17: HMM-based CNV prediction

INFO [2023-05-18 09:44:24] predict_CNV_via_

INFO [2023-05-18 10:23:59] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-05-18 10:23:59] Quantiles of plotted data range: 1,3,3,4,6
INFO [2023-05-18 10:24:05] plot_cnv_references:Start
INFO [2023-05-18 10:24:05] Reference data size: Cells= 3372 Genes= 3741
INFO [2023-05-18 10:28:46] plot_cnv_references:Number reference groups= 1
INFO [2023-05-18 10:28:46] plot_cnv_references:Plotting heatmap.
INFO [2023-05-18 10:28:53] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-05-18 10:28:53] Quantiles of plotted data range: 1,3,3,3,6
INFO [2023-05-18 10:29:02] 

	STEP 18: Run Bayesian Network Model on HMM predicted CNVs

INFO [2023-05-18 10:29:03] Creating the following Directory:  /data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF2221/alternate/infercnv_results/BayesNetOutput.HMM

INFO [2023-05-19 02:10:23] ::plot_cnv:Start
INFO [2023-05-19 02:10:23] ::plot_cnv:Current data dimensions (r,c)=3741,5455 Total=4332669.16045638 Min=0 Max=0.992477073415229.
INFO [2023-05-19 02:10:23] ::plot_cnv:Depending on the size of the matrix this may take a moment.
INFO [2023-05-19 02:10:24] plot_cnv_observation:Start
INFO [2023-05-19 02:10:24] Observation data size: Cells= 2083 Genes= 3741
INFO [2023-05-19 02:10:25] plot_cnv_observation:Writing observation groupings/color.
INFO [2023-05-19 02:10:25] plot_cnv_observation:Done writing observation groupings/color.
INFO [2023-05-19 02:10:25] plot_cnv_observation:Writing observation heatmap thresholds.
INFO [2023-05-19 02:10:25] plot_cnv_observation:Done writing observation heatmap thresholds.
INFO [2023-05-19 02:10:29] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-05-19 02:10:29] Quantiles of plotted data range: 0,0,0.66302674497